<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/Regressione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importa Libreria

In [1]:
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.graph_objects as go
# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions

import pandas as pd
from datetime import datetime

## Load Data

In [2]:
# Dati Numpy
R_X = np.linspace(-2,8,1000)
R_Y = np.exp(0.2*R_X)*np.sin(3*R_X) - 10*np.cos(R_X)

# Normalization
R_mean = np.mean(R_X)
R_std = np.std(R_X)

# Dati Pytorch Tensor
R_Xt = torch.from_numpy(R_X).type(torch.float32).reshape(-1,1)#.unsqueeze(1)
R_Yt = torch.from_numpy(R_Y).type(torch.float32).unsqueeze(1)
print(f"X Tensor data shape: ", R_Xt.shape)
print(f"Y Tensor data shape: ", R_Yt.shape)

# Training and Test Set
R_X_train, R_X_test, R_Y_train, R_Y_test = train_test_split(R_X,R_Y,test_size=0.3,shuffle=True,random_state=4)
print(f"X Train shape: {R_X_train.shape} , X Test shape: {R_X_test.shape}")


X Tensor data shape:  torch.Size([1000, 1])
Y Tensor data shape:  torch.Size([1000, 1])
X Train shape: (700,) , X Test shape: (300,)


In [3]:
# Visualization
fig = go.Figure()
fig.add_traces( go.Scatter(x=R_X, y=R_Y,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Real data") )
fig.add_traces( go.Scatter(x=R_X_train, y=R_Y_train,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Train data") )
fig.add_traces( go.Scatter(x=R_X_test, y=R_Y_test,hovertemplate='x: %{x} <br>y: %{y}',mode="markers", name="Test data") )

fig.update_layout(title="Funzione di stimare")
fig.show()

In [11]:
# Tensor Dataset Che converte i dati da numpy a Pytorch
class CustomTensorDataset(Dataset):
    def __init__(self, x,y,mean,std):
        x = (x - mean)/std
        self.x = torch.from_numpy(x).type(torch.float32).unsqueeze(1)
        self.y = torch.from_numpy(y).type(torch.float32).unsqueeze(1)
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
R_train_ds = CustomTensorDataset(R_X_train,R_Y_train,R_mean,R_std)
R_test_ds = CustomTensorDataset(R_X_test,R_Y_test,R_mean,R_std)

## Training


In [12]:
# Validation: Metric Regression
def metrics_func_regression(target, output):
  # Comptue mean squaer error (Migliora quanto piu' ci avviciniamo a zero)
  mse = torch.sum((output - target) ** 2)
  return mse

# Validation: Metric cassification
def metrics_func_classification(target, output):
  # Compute number of correct prediction
  pred = output.argmax(dim=-1,keepdim=True)
  corrects =pred.eq(target.reshape(pred.shape)).sum().item()
  return -corrects # minus for coeherence with best result is the most negative one

# Training: Loss calculation and backward step
def loss_batch(loss_func,metric_func, xb,yb,yb_h, opt=None):
  # obtain loss
  loss = loss_func(yb_h, yb)
  # obtain permormance metric 
  metric_b = metric_func(yb,yb_h)
  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()
  return loss.item(), metric_b

# Trainig: Function 1 epoch
def loss_epoch(model, loss_func,metric_func, dataset_dl, opt, device):
  loss = 0.0
  metric = 0.0
  len_data = len(dataset_dl.dataset)
  # Get batch data
  for xb,yb in dataset_dl:    
    # Send to cuda the data (batch size)
    xb = xb.to(device)
    yb = yb.to(device)
    # obtain model output 
    yb_h = model.forward(xb)
    # Loss and Metric Calculation
    loss_b, metric_b = loss_batch(loss_func,metric_func, xb,yb,yb_h,opt)
    loss += loss_b
    if metric_b is not None:
      metric+=metric_b 
  loss /=len_data
  metric /=len_data
  return loss, metric

# Training: Iterate on epochs
def train_val(epochs, model, loss_func, metric_func, opt, train_dl,test_dl,device, path2weigths="./weights.pt"):
  lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.999) #  lr = lr * gamma ** last_epoch
  best_val_metric = 1000000
  for epoch in range(epochs):
    model.train()
    train_loss,train_metric = loss_epoch(model, loss_func, metric_func,train_dl, opt,device)
    lr_scheduler.step()
    model.eval()
    with torch.no_grad():
      val_loss, val_metric = loss_epoch(model, loss_func, metric_func, test_dl,opt=None,device=device)
      print("epoch: %d, train_loss: %.6f, val loss: %.6f,  train_metric: %.3f test_metric: %.3f lr: %.5f)" % (epoch,train_loss, val_loss,train_metric,val_metric,opt.param_groups[0]['lr']))
      if (val_metric <= best_val_metric):        
        # Save Models (It save last weights)
        torch.save(model.state_dict(),path2weigths)
        best_val_metric = val_metric


In [14]:
class RegressionNet(nn.Module):
    def __init__(self,num_inputs):
        super(RegressionNet,self).__init__()
        self.fc1 = nn.Linear(num_inputs,100)
        self.fc2 = nn.Linear(100,50)
        self.fc3 = nn.Linear(50,1)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")

# Regression
R_model = RegressionNet(num_inputs=1).to(device)
R_loss_func = nn.MSELoss(reduction="sum") 
R_opt = optim.Adam(R_model.parameters(),lr=0.005)
R_train_dl = DataLoader(R_train_ds,batch_size=100,shuffle=True)
R_test_dl = DataLoader(R_test_ds,batch_size=50,shuffle=True)

# Regression
train_val(2000,R_model,R_loss_func,metrics_func_regression,R_opt, R_train_dl,R_test_dl,device,path2weigths="./weights_regression.pt")


epoch: 0, train_loss: 47.123795, val loss: 46.170017,  train_metric: 47.124 test_metric: 46.170 lr: 0.00500)
epoch: 1, train_loss: 44.748514, val loss: 44.297085,  train_metric: 44.749 test_metric: 44.297 lr: 0.00499)
epoch: 2, train_loss: 42.754157, val loss: 41.411383,  train_metric: 42.754 test_metric: 41.411 lr: 0.00499)
epoch: 3, train_loss: 39.174116, val loss: 36.679178,  train_metric: 39.174 test_metric: 36.679 lr: 0.00498)
epoch: 4, train_loss: 34.326625, val loss: 30.778174,  train_metric: 34.327 test_metric: 30.778 lr: 0.00498)
epoch: 5, train_loss: 28.257298, val loss: 24.004098,  train_metric: 28.257 test_metric: 24.004 lr: 0.00497)
epoch: 6, train_loss: 22.149866, val loss: 18.480194,  train_metric: 22.150 test_metric: 18.480 lr: 0.00497)
epoch: 7, train_loss: 17.934423, val loss: 14.489844,  train_metric: 17.934 test_metric: 14.490 lr: 0.00496)
epoch: 8, train_loss: 15.173620, val loss: 12.198866,  train_metric: 15.174 test_metric: 12.199 lr: 0.00496)
epoch: 9, train_los

## Test

In [15]:
# Run on cpu
device = torch.device("cpu")

# Load Regression
R_model = RegressionNet(num_inputs=1)
R_weights = torch.load("weights_regression.pt")
R_model.load_state_dict(R_weights)
R_model = R_model.to(device)

# Predict Regression
R_Xt = torch.from_numpy(R_X).type(torch.float32).unsqueeze(1)
R_Xt = (R_Xt - R_mean) / R_std
R_Y_hat = R_model.forward(R_Xt).detach().numpy().reshape(-1)

# Visualize Regression
fig = go.Figure()
fig.add_traces( go.Scatter(x=R_X, y=R_Y, name="Real",hovertemplate='x: %{x} <br>y: %{y}') )
fig.add_traces( go.Scatter(x=R_X, y=R_Y_hat, name="Predicted",hovertemplate='x: %{x} <br>y: %{y}') )
fig.update_layout(title="Regression Results",hovermode="x")
fig.show()